Hàm xử lý dữ liệu đầu vào.

In [ ]:
import os
import io
import urllib.request as reqest
import numpy as np
import pandas as pd
import tensorflow as tf

#Khai báo params
IRIS_TRAIN = 'iris_train.csv'
IRIS_TRAIN_PATH = 'http://download.tensorflow.org/data/iris_training.csv'
IRIS_TEST

def get_data(is_train_data = True):
    